In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
!pip install scikit-learn


   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.7 MB 996.7 kB/s eta 0:00:09
   -- ------------------------------------- 0.5/8.7 MB 996.7 kB/s eta 0:00:09
   --- ------------------------------------ 0.8/8.7 MB 906.1 kB/s eta 0:00:09
   ---- ----------------------------------- 1.0/8.7 MB 887.8 kB/s eta 0:00:09
   ---- ----------------------------------- 1.0/8.7 MB 887.8 kB/s eta 0:00:09
   ------- -------------------------------- 1.6/8.7 MB 956.9 kB/s eta 0:00:08
   ------- -------------------------------- 1.6/8.7 MB 956.9 kB/s eta 0:00:08
   -------- ------------------------------- 1.8/8.7 MB 938.7 kB/s eta 0:00:08
   --------- ------------------------------ 2.1/8.7 MB 946.9 kB/s eta 0:00:07
   ---------- --------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## ***Chuẩn bị dữ liệu***


In [9]:
import pandas as pd
df = pd.read_excel(r"C:\Users\Dell\Downloads\Weekly_pivot_with_quarter_season.xlsx")
print(df.shape)
print(df.head())
print(df.info())


(52, 8)
   Week_number  Product_0979  Product_0993  Product_1157  Product_1159  \
0            1          4700             0         10000         50000   
1            2         11800             0        150000         10000   
2            3         16400           100        130000        130000   
3            4          5200             0         40000         30000   
4            5          8000             0         10000         60000   

   Product_1938  Quarter    Season  
0            19        1  mùa Đông  
1            11        1  mùa Đông  
2            10        1  mùa Đông  
3             8        1  mùa Đông  
4            11        1  mùa Đông  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Week_number   52 non-null     int64 
 1   Product_0979  52 non-null     int64 
 2   Product_0993  52 non-null     int64 
 3   Product_

##  ***Tiền xử lí cơ bản***

- Xử lý thiếu (na), kiểu dữ liệu, chuyển Season thành one-hot, để Quarter giữ nguyên (hoặc one-hot).


- Tạo biến thời gian (week index) nếu cần.

In [11]:
# copy
data = df.copy()

# bỏ dòng có missing (hoặc impute tuỳ dữ liệu)
data = data.dropna()  # hoặc dùng fillna nếu hợp lý

# one-hot cho Season (drop_first để tránh multicollinearity)
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse_output=False, drop='first')
season_ohe = enc.fit_transform(data[['Season']])
season_cols = enc.get_feature_names_out(['Season'])
season_df = pd.DataFrame(season_ohe, columns=season_cols, index=data.index)

# optional: one-hot cho Quarter (nếu quý không phải thứ tự tuyến tính)
quarter_ohe = pd.get_dummies(data['Quarter'].astype(str), prefix='Q', drop_first=True)

# gộp
data = pd.concat([data.drop(columns=['Season','Quarter']), season_df, quarter_ohe], axis=1)


## ***Tạo đặc trưng thời gian*** (lag features & rolling)

Thường dữ liệu time-series cần lag để mô hình tuyến tính học được phụ thuộc thời gian.

In [12]:
products = ["Product_0979","Product_0993","Product_1157","Product_1159","Product_1938"]

# tạo lag 1, lag 2 cho từng product
for p in products:
    data[f"{p}_lag1"] = data[p].shift(1)
    data[f"{p}_lag2"] = data[p].shift(2)
    data[f"{p}_roll4_mean"] = data[p].rolling(window=4, min_periods=1).mean().shift(1)

# drop đầu có NaN do shift
data = data.dropna().reset_index(drop=True)


## ***Chọn X và y***

- X bao gồm: Week_number, các one-hot season/quarter, lag/rolling features.


- y là 5 cột sản phẩm.

In [13]:
X = data.drop(columns=products)
y = data[products]


## ***Chia train/test*** (theo time series hoặc random)

- Nếu dữ liệu tuần thời gian, tốt nhất chia theo thời gian (train cũ → test mới).


- Nếu không có thứ tự mạnh, dùng train_test_split.

In [14]:
split_idx = int(len(data)*0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(40, 22) (10, 22) (40, 5) (10, 5)


### ***Chuẩn hóa dữ liệu*** ###

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
num_cols = X_train.select_dtypes(include=['int64','float64']).columns
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])


C:\Users\Dell\AppData\Local\Temp\ipykernel_22988\303616938.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
C:\Users\Dell\AppData\Local\Temp\ipykernel_22988\303616938.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[num_cols] = scaler.transform(X_test[num_cols])


## ***Huấn luyện mô hình Linear Regression đa biến cho từng sản phẩm***

In [16]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


## ***Dự đoán & đánh giá***

- Metricts: R², RMSE, MAE cho từng sản phẩm.


- Kiểm tra residuals (độ phân bố phần dư), scatter plot, và histogram.

In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Dự đoán
y_pred = model.predict(X_test)

# Tạo dict để lưu kết quả
results = {}

for i, col in enumerate(y.columns):
    y_true = y_test[col]
    y_p = y_pred[:, i]

    # Tính metrics cơ bản
    r2 = r2_score(y_true, y_p)
    rmse = np.sqrt(mean_squared_error(y_true, y_p))
    mae = mean_absolute_error(y_true, y_p)

    # Tính MAPE (loại bỏ giá trị 0 để tránh chia cho 0)
    y_true_nonzero = y_true[y_true != 0]
    y_p_nonzero = y_p[y_true != 0]

    if len(y_true_nonzero) > 0:
        mape = np.mean(np.abs((y_true_nonzero - y_p_nonzero) / y_true_nonzero)) * 100
    else:
        mape = np.nan

    results[col] = {
        'R2': r2,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE (%)': mape
    }

# Chuyển thành DataFrame để hiển thị đẹp
results_df = pd.DataFrame(results).T
pd.set_option("display.float_format", lambda x: f"{x:.4f}")
print(results_df)



                   R2        RMSE         MAE  MAPE (%)
Product_0979  -2.2946  13094.9797   9011.8784   74.3185
Product_0993 -32.0722    517.5762    457.9335  296.2318
Product_1157 -28.8058 413948.5306 359670.8408  973.2331
Product_1159 -22.5488 116465.1348 105233.1538  616.4802
Product_1938 -16.7898     34.6297     27.6335  282.6280
